# Task 1


In [5]:
# importing libraries
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from gensim.models import Word2Vec
import numpy as np
import string
import time

In [6]:
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
# Preprocessing of the data

st = time.time()

cfp = './data/Training-dataset.csv'
filed = pd.read_csv(cfp)
data = filed['plot_synopsis'].tolist()

def tagfunc(nltk_tag):
  if nltk_tag.startswith('J'):
    return wordnet.ADJ
  elif nltk_tag.startswith('V'):
    return wordnet.VERB
  elif nltk_tag.startswith('N'):
    return wordnet.NOUN
  elif nltk_tag.startswith('R'):
    return wordnet.ADV
  return None

def lemfunc(w):
  lemmatizer = WordNetLemmatizer()
  tag=nltk.pos_tag([w])[0]
  wordnet_tagged = [tag[0], tagfunc(tag[1])]
  desc=wordnet_tagged[1]
  if desc != None:
    return (lemmatizer.lemmatize(w,desc))
  else:
    return (lemmatizer.lemmatize(w))

# tokenization
def tokenfunc(i):
  t = nltk.word_tokenize(i)
  return t

def filteringfunc(w, stop_words):
  ft = []
  for word in w:
    if word.isalpha() and word.lower() not in stop_words:
      ft.append(lemfunc(word))
  return ft





custom_stopwords = {'movie', 'film', 'story', 'character','cinema'}
stop_words = set(stopwords.words('english'))
stop_words.update(custom_stopwords)
final2 = []
for i in data:
  final2.append(filteringfunc(tokenfunc(i), stop_words))

final1 = final2
filed['_processed'] = [' '.join(tokens) for tokens in final1]

print(filed['_processed'])

0       recent amount challenge Billy Lo Bruce Lee fri...
1       city Tremont renowned investigative journalist...
2       Lankester Merrin veteran Catholic priest exorc...
3       Serendipity Seasons heartwarming romantic tell...
4       Young naive slacker Adam Jack life mum Nicky K...
                              ...                        
8252    serve eight month sentence break enter Git Hyn...
8253    Mystery crew head Chicago talent show call Tal...
8254    run Another Life revolve around Davidson famil...
8255    North Bend Psychiatric Hospital young patient ...
8256    depiction various scene usually violent bizarr...
Name: _processed, Length: 8257, dtype: object


**Method a**

Tf*idf

In [8]:
vectorizer = TfidfVectorizer(ngram_range = (1,3))
vec = vectorizer.fit_transform(filed['_processed'])
feature_names =vectorizer.get_feature_names_out()
print(feature_names)
print(len(feature_names))

['aa' 'aa another' 'aa another storyline' ... '齐天大圣' '齐天大圣 traditional'
 '齐天大圣 traditional chinese']
5788190


In [9]:
#Input data
def datainput(f):
    return pd.read_csv(f, header = None)

# Calculating cosine similarity
def calculate_cosine_similarity(t1, t2, vec, vectorizer):
    vocab = vectorizer.vocabulary_
    idxt1, idxt2 = vocab.get(t1), vocab.get(t2)
    if None not in (idxt1, idxt2):
        v1, v2 = vec.getcol(idxt1).toarray(), vec.getcol(idxt2).toarray()
        s = np.dot(v1.T, v2)[0, 0] / (np.linalg.norm(v1) * np.linalg.norm(v2))
        return s
    return 0

# Calculating similarities for terms
def calculate_similarity_for_terms(vdf, vec, vectorizer):
    stfidf = vdf.apply(lambda row: (
        row.iloc[0],
        row.iloc[1],
        row.iloc[2],
        calculate_cosine_similarity(row.iloc[1], row.iloc[2], vec, vectorizer)
    ), axis=1).tolist()
    return stfidf

# Displaying result
def display_results(stfidf):
    print("ID\tTerm Pair\t\tCalculated Cosine Similarity (TF-IDF)")
    for ID, t1, t2, s in stfidf:
      print(f"{ID}\t{t1} - {t2}:\t\t{s}")

# Saving the result to csv file
def save_results_to_csv(sc, ocsvp):
    sc.to_csv(ocsvp, header=False, index=False)

# Paths
vdp = './data/Task-1-validation-dataset.csv'
ocsvp = './data/10946159-Task1-method-a-validation.csv'

# Read dataset
vdf = datainput(vdp)

# Calculate cosine similarity
stfidf = calculate_similarity_for_terms(vdf, vec, vectorizer)

# Display and save results
display_results(stfidf)
rtfidfvdf = pd.DataFrame(stfidf, columns=['ID', 'Term1', 'Term2', 'Cosine_Similarity_TFIDF'])
sc = rtfidfvdf[['ID', 'Cosine_Similarity_TFIDF']]
save_results_to_csv(sc, ocsvp)

ID	Term Pair		Calculated Cosine Similarity (TF-IDF)
1	absorb - learn:		0.03786242115499603
2	absorb - withdraw:		0.0
3	achieve - accomplish:		0.050486947521321296
4	achieve - try:		0.05635017195989093
6	acquire - get:		0.05493827471273253
7	acquire - obtain:		0.0735409232160422
8	acquire - find:		0.09543009003255346
11	apple - sauce:		0.0
12	apple - lemon:		0.0003837010706794071
13	apple - sunshine:		0.0
14	area - region:		0.10408029773504186
16	area - corner:		0.0869966522210333
17	arm - shoulder:		0.11470456841877806
20	arm - body:		0.21405862661811972
21	arm - vein:		0.021740745420710427
22	arm - knee:		0.05126181982685467
23	arm - bone:		0.057906913732136106
24	arm - neck:		0.15153934740492933
25	ball - cannon:		0.013354605073064163
26	ball - basket:		0.019400308712406153
29	bath - trick:		0.03390900513881301
30	bath - wife:		0.05234458823866435
31	bath - balloon:		0.011171531223867846
32	bed - bedroom:		0.2350906371508938
33	bed - blanket:		0.06815054053116326
34	bed - crib:		0.01

In [10]:
def datainput(f):
    return pd.read_csv(f, header = None)

def calculate_cosine_similarity(t1, t2, vec, vectorizer):
    vocab = vectorizer.vocabulary_
    idxt1, idxt2 = vocab.get(t1), vocab.get(t2)
    if None not in (idxt1, idxt2):
        v1, v2 = vec.getcol(idxt1).toarray(), vec.getcol(idxt2).toarray()
        s = np.dot(v1.T, v2)[0, 0] / (np.linalg.norm(v1) * np.linalg.norm(v2))
        return s
    return 0

def calculate_similarity_for_terms(vdf, vec, vectorizer):
    stfidf = vdf.apply(lambda row: (
        row.iloc[0],
        row.iloc[1],
        row.iloc[2],
        calculate_cosine_similarity(row.iloc[1], row.iloc[2], vec, vectorizer)
    ), axis=1).tolist()
    return stfidf

def display_results(stfidf):
    print("ID\tTerm Pair\t\tCalculated Cosine Similarity (TF-IDF)")
    for ID, t1, t2, s in stfidf:
      print(f"{ID}\t{t1} - {t2}:\t\t{s}")

def save_results_to_csv(sc, ocsvp):
    sc.to_csv(ocsvp, header=False, index=False)

# Paths
vdp = './data/Task-1-test-dataset2.csv'
ocsvp = './data/10946159-Task1-method-a.csv'

# Read dataset
vdf = datainput(vdp)

# Calculate cosine similarity
stfidf = calculate_similarity_for_terms(vdf, vec, vectorizer)

# Display and save results
display_results(stfidf)
rtfidfvdf = pd.DataFrame(stfidf, columns=['ID', 'Term1', 'Term2', 'Cosine_Similarity_TFIDF'])
sc = rtfidfvdf[['ID', 'Cosine_Similarity_TFIDF']]
save_results_to_csv(sc, ocsvp)

ID	Term Pair		Calculated Cosine Similarity (TF-IDF)
12	bad - awful:		0.0391088933952414
54	bad - terrific:		0.004032206827607899
332	beach - island:		0.16419938660300418
422	beach - reef:		0.05286928921722972
557	car - bridge:		0.11379895981934447
499	car - carriage:		0.048925440372529695
256	door - gate:		0.0818923006733763
568	door - kitchen:		0.24761171484334552
20	hard - tough:		0.037391212867670186
15	hard - simple:		0.028911913969018137
438	house - carpet:		0.05960009172213288
555	house - barn:		0.12427732963235442
2028	journey - voyage:		0.030867652386844507
440	journey - conquest:		0.010279588380179495
908	keep - protect:		0.1206418540915078
959	keep - give:		0.3493568795217225
43	large - big:		0.09917181832221897
110	large - flexible:		0.015545788650896055
793	leave - enter:		0.26440152797297867
823	leave - go:		0.5438219544949887
733	task - highway:		0.014673676694478595
542	task - job:		0.054985662784213234
35	weird - odd:		0.009642581364955731
34	weird - normal:		0.01865311

In [11]:
#!python task1_eval_script_student_version.py 10946159-Task1-method-a-validation.csv Task-1-validation-dataset.csv

The following simalarity scores may need checking:
(achieve,accomplish) similarity score: 0.050486947521321296, gold ranking: 8.57
(achieve,try) similarity score: 0.05635017195989093, gold ranking: 4.42
----------------------------
(acquire,get) similarity score: 0.05493827471273253, gold ranking: 8.82
(acquire,obtain) similarity score: 0.0735409232160422, gold ranking: 8.57
----------------------------
(acquire,get) similarity score: 0.05493827471273253, gold ranking: 8.82
(acquire,find) similarity score: 0.09543009003255346, gold ranking: 6.38
----------------------------
(acquire,obtain) similarity score: 0.0735409232160422, gold ranking: 8.57
(acquire,find) similarity score: 0.09543009003255346, gold ranking: 6.38
----------------------------
(apple,sauce) similarity score: 0.0, gold ranking: 1.43
(apple,sunshine) similarity score: 0.0, gold ranking: 0.58
----------------------------
(arm,shoulder) similarity score: 0.11470456841877806, gold ranking: 4.85
(arm,body) similarity scor

In [12]:
et = time.time()
print(et - st)


483.1431133747101


**Method b**

Word2Vec


In [13]:
stime=time.time()
filed['processed_data_task2'] = final2
print(filed['processed_data_task2'])

0       [recent, amount, challenge, Billy, Lo, Bruce, ...
1       [city, Tremont, renowned, investigative, journ...
2       [Lankester, Merrin, veteran, Catholic, priest,...
3       [Serendipity, Seasons, heartwarming, romantic,...
4       [Young, naive, slacker, Adam, Jack, life, mum,...
                              ...                        
8252    [serve, eight, month, sentence, break, enter, ...
8253    [Mystery, crew, head, Chicago, talent, show, c...
8254    [run, Another, Life, revolve, around, Davidson...
8255    [North, Bend, Psychiatric, Hospital, young, pa...
8256    [depiction, various, scene, usually, violent, ...
Name: processed_data_task2, Length: 8257, dtype: object


In [14]:
word2vec_model = Word2Vec(sentences=filed['processed_data_task2'], vector_size=270, window=17, min_count=1, workers=4)


In [15]:
dense_representations = []

for index, row in filed.iterrows():
    synopsis_tokens = word_tokenize(row['plot_synopsis'].lower())
    valid_tokens = [word for word in synopsis_tokens if word in word2vec_model.wv]

    if valid_tokens:
        mean_representation = np.mean([word2vec_model.wv[word] for word in valid_tokens], axis=0)
        dense_representations.append(mean_representation)
    else:
        dense_representations.append(None)

filed['dense_representation'] = dense_representations


In [16]:

def calculate_cosine_similarity(t1, t2, model):
    v1 = []
    v2 = []

    # Populate vectors1
    for w in word_tokenize(t1.lower()):
        if w in model.wv:
            v1.append(model.wv[w])

    # Populate vectors2
    for w in word_tokenize(t2.lower()):
        if w in model.wv:
            v2.append(model.wv[w])

    # Check if vectors are empty (no words in the vocabulary)
    if len(v1) == 0 or len(v2) == 0:
        return 0

    # Calculate mean vectors
    v1 = np.mean(v1, axis=0, keepdims=True)
    v2 = np.mean(v2, axis=0, keepdims=True)

    s = cosine_similarity(v1, v2)
    return s[0][0]


def calculate_similarity_for_terms(vdf, model):
    def calculate_similarity(row):
        t1, t2, ID = row[1], row[2], row[0]
        s = calculate_cosine_similarity(t1, t2, model)
        return ID, t1, t2, s

    sword2vec = vdf.apply(lambda row: calculate_similarity(row), axis=1).tolist()
    return sword2vec

def display_results(sword2vec):
    print("ID\tTerm Pair\t\tCalculated Cosine Similarity")
    for ID, t1, t2, s in sword2vec:
        print(f"{ID}\t{t1} - {t2}:\t\t{s}")

def save_results_to_csv(selected_columns, output_csv_path):
    selected_columns.to_csv(output_csv_path, header=False, index=False)

# Load Word2Vec model
# word2vec_model = Word2Vec(sentences=df['processed_data_task2'], vector_size=300, window=20, min_count=1, workers=4)

# Paths
vdp = './data/Task-1-validation-dataset.csv'
ocsvp = './data/10946159-Task1-method-b-validation.csv'

# Read dataset
vdf = pd.read_csv(vdp, header=None)

# Calculate cosine similarity
sword2vec = calculate_similarity_for_terms(vdf, word2vec_model)

# Display and save results
display_results(sword2vec)
rdf = pd.DataFrame(sword2vec, columns=['ID', 'Term1', 'Term2', 'Cosine_Similarity'])
sc = rdf[['ID', 'Cosine_Similarity']]
save_results_to_csv(sc, ocsvp)









ID	Term Pair		Calculated Cosine Similarity
1	absorb - learn:		0.22451701760292053
2	absorb - withdraw:		0.2781107723712921
3	achieve - accomplish:		0.8310285210609436
4	achieve - try:		-0.044793419539928436
6	acquire - get:		-0.13899172842502594
7	acquire - obtain:		0.8062546253204346
8	acquire - find:		-0.008534951135516167
11	apple - sauce:		0.7388456463813782
12	apple - lemon:		0.6387547850608826
13	apple - sunshine:		0.5693299770355225
14	area - region:		0.5164653062820435
16	area - corner:		0.23328733444213867
17	arm - shoulder:		0.7193329334259033
20	arm - body:		0.2898249626159668
21	arm - vein:		0.43273162841796875
22	arm - knee:		0.6335611343383789
23	arm - bone:		0.4884495139122009
24	arm - neck:		0.6589184999465942
25	ball - cannon:		0.4123767614364624
26	ball - basket:		0.5135512948036194
29	bath - trick:		0.2352110743522644
30	bath - wife:		-0.01104647759348154
31	bath - balloon:		0.48851826786994934
32	bed - bedroom:		0.8196549415588379
33	bed - blanket:		0.67429709434509

In [17]:

def calculate_cosine_similarity(t1, t2, model):
    v1 = []
    v2 = []

    # Populate vectors1
    for w in word_tokenize(t1.lower()):
        if w in model.wv:
            v1.append(model.wv[w])

    # Populate vectors2
    for w in word_tokenize(t2.lower()):
        if w in model.wv:
            v2.append(model.wv[w])

    # Check if vectors are empty (no words in the vocabulary)
    if len(v1) == 0 or len(v2) == 0:
        return 0

    # Calculate mean vectors
    v1 = np.mean(v1, axis=0, keepdims=True)
    v2 = np.mean(v2, axis=0, keepdims=True)

    s = cosine_similarity(v1, v2)
    return s[0][0]


def calculate_similarity_for_terms(vdf, model):
    def calculate_similarity(row):
        t1, t2, ID = row[1], row[2], row[0]
        s = calculate_cosine_similarity(t1, t2, model)
        return ID, t1, t2, s

    sword2vec = vdf.apply(lambda row: calculate_similarity(row), axis=1).tolist()
    return sword2vec

def display_results(sword2vec):
    print("ID\tTerm Pair\t\tCalculated Cosine Similarity")
    for ID, t1, t2, s in sword2vec:
        print(f"{ID}\t{t1} - {t2}:\t\t{s}")

def save_results_to_csv(selected_columns, output_csv_path):
    selected_columns.to_csv(output_csv_path, header=False, index=False)

# Load Word2Vec model
# word2vec_model = Word2Vec(sentences=df['processed_data_task2'], vector_size=300, window=20, min_count=1, workers=4)

# Paths
vdp = './data/Task-1-test-dataset2.csv'
ocsvp = './data/10946159-Task1-method-b.csv'

# Read dataset
vdf = pd.read_csv(vdp, header=None)

# Calculate cosine similarity
sword2vec = calculate_similarity_for_terms(vdf, word2vec_model)

# Display and save results
display_results(sword2vec)
rdf = pd.DataFrame(sword2vec, columns=['ID', 'Term1', 'Term2', 'Cosine_Similarity'])
sc = rdf[['ID', 'Cosine_Similarity']]
save_results_to_csv(sc, ocsvp)









ID	Term Pair		Calculated Cosine Similarity
12	bad - awful:		0.5817447900772095
54	bad - terrific:		0.16335739195346832
332	beach - island:		0.4324498176574707
422	beach - reef:		0.6170638203620911
557	car - bridge:		0.41875916719436646
499	car - carriage:		0.3410727381706238
256	door - gate:		0.6085649132728577
568	door - kitchen:		0.5580978393554688
20	hard - tough:		0.4114333689212799
15	hard - simple:		0.4209998846054077
438	house - carpet:		0.18585965037345886
555	house - barn:		0.6038413047790527
2028	journey - voyage:		0.4685031473636627
440	journey - conquest:		0.4022118151187897
908	keep - protect:		0.30095040798187256
959	keep - give:		0.3393365144729614
43	large - big:		0.32745596766471863
110	large - flexible:		0.45641404390335083
793	leave - enter:		0.3476051688194275
823	leave - go:		0.40744030475616455
733	task - highway:		-0.22484353184700012
542	task - job:		0.22510965168476105
35	weird - odd:		0.7753371596336365
34	weird - normal:		0.30384862422943115
101	glass - cryst

In [18]:
#!python task1_eval_script_student_version.py 10946159-Task1-method-b-validation.csv Task-1-validation-dataset.csv

The following simalarity scores may need checking:
(absorb,learn) similarity score: 0.22451701760292053, gold ranking: 5.48
(absorb,withdraw) similarity score: 0.2781107723712921, gold ranking: 2.97
----------------------------
(acquire,get) similarity score: -0.13899172842502594, gold ranking: 8.82
(acquire,obtain) similarity score: 0.8062546253204346, gold ranking: 8.57
----------------------------
(acquire,get) similarity score: -0.13899172842502594, gold ranking: 8.82
(acquire,find) similarity score: -0.008534951135516167, gold ranking: 6.38
----------------------------
(apple,sauce) similarity score: 0.7388456463813782, gold ranking: 1.43
(apple,lemon) similarity score: 0.6387547850608826, gold ranking: 4.05
----------------------------
(arm,body) similarity score: 0.2898249626159668, gold ranking: 4.05
(arm,vein) similarity score: 0.43273162841796875, gold ranking: 3.65
----------------------------
(arm,body) similarity score: 0.2898249626159668, gold ranking: 4.05
(arm,knee) sim

In [19]:
et = time.time()
print(et - st)

582.355302810669
